Preprocessing list

EndInvFinal:
- `size`: 
    - remove string, uniform the units (there are ml and Liter)
    - rows contains ml --> remove 'ml' strings 
    - rows contains Liter or L--> divide by 1000
    - 50mL 3 Pk --> 